In [ ]:
import pandas as pd
import requests
import plotly.express as px
from ipywidgets import interact, widgets
import numpy as np
import json
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets
import csv
import base64
from io import BytesIO
import sys
from io import StringIO
from PIL import Image
import matplotlib.pyplot as plt

csv.field_size_limit(10**7)

dropbox_url = "https://www.dropbox.com/scl/fi/sgoddzdeagwauz7nf24ei/info.csv?rlkey=49brtovllwvz7gdw6zqsw6bsn&st=yc0j0dgu&dl=1"

response = requests.get(dropbox_url)

if response.status_code == 200:
    try:
        print('status=200')
        df = pd.read_csv(
            StringIO(response.content.decode('utf-8')),
            sep=None,
            engine="python",
            quoting=csv.QUOTE_MINIMAL,
            skipinitialspace=True
        )
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Error reading CSV: {e}")
        df = pd.DataFrame() 
else:
    print("Failed to download the file from Dropbox.")
    df = pd.DataFrame()

df = df.drop(['Unnamed: 0'], axis=1)

def transform_smotenc(value):
    if isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return value
    return value  

df['SMOTENC'] = df['SMOTENC'].apply(transform_smotenc)
df['Dropout'] = pd.to_numeric(df['Dropout'], errors='coerce')
df['history'] = df['history'].apply(lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x)


In [ ]:
def create_base64_placeholder():
    img = Image.new('RGB', (100, 100), color=(255, 0, 0))  # Create a red square
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()


data = df


def decode_base64_image(encoded_str):
    decoded = base64.b64decode(encoded_str)
    image = Image.open(BytesIO(decoded))
    return image

import matplotlib.gridspec as gridspec

def plot_combined(history, tsne_encoded, cm_encoded):
    val_loss = history['val_loss']
    train_loss = history['loss']
    
    val_accuracy = history['val_accuracy']
    train_accuracy = history['accuracy']
    
    val_auc = history['val_auc']
    train_auc = history['auc']

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.plot(val_loss, label='Validation Loss', color='red')
    plt.plot(train_loss, label='Train Loss', color='black')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0.17, 0.38)
    plt.yticks(np.arange(0.17, 0.38, 0.01))
    plt.grid()
    plt.legend(loc='center right')

    plt.subplot(1, 3, 2)
    plt.plot(val_accuracy, label='Validation Accuracy', color='blue')
    plt.plot(train_accuracy, label='Train Accuracy', color='black')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(0.75, 0.96)
    plt.yticks(np.arange(0.75, 0.96, 0.01))
    plt.grid()
    plt.legend(loc='center right')

    plt.subplot(1, 3, 3)
    plt.plot(val_auc, label='Validation AUC', color='green')
    plt.plot(train_auc, label='Train AUC', color='black')
    plt.title('AUC')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.ylim(0.75, 0.96)
    plt.yticks(np.arange(0.75, 0.96, 0.01))
    plt.grid()
    plt.legend(loc='center right')

    plt.tight_layout()
    plt.show()

    tsne_image = decode_base64_image(tsne_encoded)
    cm_image = decode_base64_image(cm_encoded)
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(tsne_image)
    axes[0].set_title("t-SNE")
    axes[0].axis("off")
    
    axes[1].imshow(cm_image)
    axes[1].set_title("Confusion Matrix")
    axes[1].axis("off")
    
    plt.tight_layout()
    plt.show()




def interactive_plot(isol_forest, iqr, box_cox, robust_scaler, normalization, SMOTENC, one_hot, Dropout, BatchNorm):
    filtered_row = data[
        (data['isol_forest'] == isol_forest) &
        (data['iqr'] == iqr) &
        (data['box_cox'] == box_cox) &
        (data['robust_scaler'] == robust_scaler) &
        (data['normalization'] == normalization) &
        (data['BatchNorm'] == BatchNorm)
        (data['one_hot'] == one_hot) &
        (data['SMOTENC'] == SMOTENC) &
        (data['Dropout'] == Dropout) 
    ]
    
    if filtered_row.empty:
        print("No matching data for the selected combination.")
        return
    
    history = filtered_row['history'].iloc[0]
    tsne_encoded = filtered_row['t-SNE'].iloc[0]
    cm_encoded = filtered_row['confusion_matrix'].iloc[0]
    
    plot_combined(history, tsne_encoded, cm_encoded)


isol_forest = widgets.Checkbox(value=True, description='isol_forest')
iqr = widgets.Checkbox(value=True, description='iqr')
box_cox = widgets.Checkbox(value=True, description='box_cox')
robust_scaler = widgets.Checkbox(value=True, description='robust_scaler')
BatchNorm = widgets.Checkbox(value=True, description='BatchNorm')
normalization = widgets.Dropdown(
        options=['False', 'normalization', 'standardization'], value='False', description='normalization'
    )

one_hot = widgets.RadioButtons(
        options=['label_enc', 'one_hot'], value='label_enc', description='Encoding:',
        style={'description_width': 'initial'}
    )

SMOTENC = widgets.SelectionSlider(
        options=['False', 0.4, 0.6, 0.8],
        value='False',
        description='SMOTENC',
        continuous_update=False
    )
Dropout = widgets.SelectionSlider(
        options=[0, 0.15, 0.25, 0.4],
        value=0,
        description='Dropout',
        continuous_update=False
    )

@interact(
    isol_forest=isol_forest,
    iqr=iqr,
    box_cox=box_cox,
    robust_scaler=robust_scaler,
    BatchNorm=BatchNorm,
    normalization=normalization,
    one_hot=one_hot,
    SMOTENC=SMOTENC,
    Dropout=Dropout
)
def update_plot(isol_forest, iqr, box_cox, robust_scaler, BatchNorm, normalization, one_hot, SMOTENC, Dropout):
    filtered_row = data[
        (data['isol_forest'] == isol_forest) &
        (data['iqr'] == iqr) &
        (data['box_cox'] == box_cox) &
        (data['robust_scaler'] == robust_scaler) &
        (data['BatchNorm'] == BatchNorm) &
        (data['normalization'] == normalization) &
        (data['one_hot'] == one_hot) & 
        (data['SMOTENC'] == SMOTENC) &
        (data['Dropout'] == Dropout)
    ]
    
    if filtered_row.empty:
        print("No matching data for the selected combination.")
        return
    
    history = filtered_row['history'].iloc[0]
    tsne_encoded = filtered_row['t-SNE'].iloc[0]
    cm_encoded = filtered_row['confusion_matrix'].iloc[0]
    
    plot_combined(history, tsne_encoded, cm_encoded)



